In [1]:
# Import login values from config
from configparser import *

In [2]:
# Import packages needed
from snowflake.snowpark import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from snowflake.snowpark import Window
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

In [3]:
import sys, json
with open("connection_parameters_prod.json") as jsonfile:
    credentials_dict = json.load(jsonfile)

In [4]:
def snowpark_session_create():
    session = Session.builder.configs(credentials_dict).create()
    return session

In [5]:
demo_session = snowpark_session_create()

In [7]:
demo_session.use_database("SNF_MDRAZ")
demo_session.use_schema("bronze")

In [18]:
kna = demo_session.sql("select * from snf_mdraz.staging.kna1")

In [23]:
df = kna[['INDEX','KUNNR','GENDE','NAME1','NAME2','STRAS','ORT01','PSTLZ','LAND1','LAND2','RSCON','TELF1','REGIO','BIRTH','LATI','LONG']]

In [24]:
df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"INDEX"    |"KUNNR"   |"GENDE"  |"NAME1"   |"NAME2"     |"STRAS"                  |"ORT01"            |"PSTLZ"   |"LAND1"  |"LAND2"         |"RSCON"                             |"TELF1"        |"REGIO"  |"BIRTH"     |"LATI"     |"LONG"     |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1172351.0  |C1272351  |Female   |Emina     |Schulting   |Via Enrico Fermi 86      |Rubiana            |10040     |IT       |Italy           |EminaSchulting@fleckens.hu          |0345 1348434   |39.0     |1969/01/24  |44.765174  |7.875138   |
|1172353.0  |C1272353  |NULL

In [37]:
df = kna.select(
    "index","kunnr","GENDE","NAME1","NAME2","STRAS","ORT01","PSTLZ","LAND1","LAND2","RSCON","TELF1","REGIO","BIRTH","LATI","LONG"
    )

In [38]:
df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"INDEX"    |"KUNNR"   |"GENDE"  |"NAME1"   |"NAME2"     |"STRAS"                  |"ORT01"            |"PSTLZ"   |"LAND1"  |"LAND2"         |"RSCON"                             |"TELF1"        |"REGIO"  |"BIRTH"     |"LATI"     |"LONG"     |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1172351.0  |C1272351  |Female   |Emina     |Schulting   |Via Enrico Fermi 86      |Rubiana            |10040     |IT       |Italy           |EminaSchulting@fleckens.hu          |0345 1348434   |39.0     |1969/01/24  |44.765174  |7.875138   |
|1172353.0  |C1272353  |NULL

In [51]:
df = df.fillna({"index":-99,"KUNNR":"NA"})

In [ ]:
df['inde']

In [53]:
print(df.dtypes)

[('INDEX', 'double'), ('KUNNR', 'string(16777216)'), ('GENDE', 'string(16777216)'), ('NAME1', 'string(16777216)'), ('NAME2', 'string(16777216)'), ('STRAS', 'string(16777216)'), ('ORT01', 'string(16777216)'), ('PSTLZ', 'string(16777216)'), ('LAND1', 'string(16777216)'), ('LAND2', 'string(16777216)'), ('RSCON', 'string(16777216)'), ('TELF1', 'string(16777216)'), ('REGIO', 'double'), ('BIRTH', 'string(16777216)'), ('LATI', 'double'), ('LONG', 'double')]


In [ ]:
#row by row insert, very slow
session.sql("BEGIN TRANSACTION")
try:
        for row in df_source.iterrows():
                insert_sql = f"""
                    insert into snf_mdraz.bronze.kna1 (INDEX,KUNNR,GENDE,NAME1,NAME2,STRAS,ORT01,PSTLZ,LAND1,LAND2,RSCON,TELF1,REGIO,BIRTH,LATI,LONG,LASTMODIFIED) 
                    VALUES({row[0]},
                            '{row['KUNNR']}',
                            '{row['GENDE']}',
                            '{row['NAME1']}',
                            '{row['NAME2']}',
                            '{row['STRAS']}',
                            '{row['ORT01']}',
                            '{row['PSTLZ']}',
                            '{row['LAND1']}',
                            '{row['LAND2']}',
                            '{row['RSCON']}',
                            '{row['TELF1']}',
                            '{row['REGIO']}',
                            '{row['BIRTH']}',
                            '{row['LATI']}',
                            '{row['LONG']}',
                            '{row['LASTMODIFIED']}')
                    """
                session.sql(insert_sql).collect()
        session.sql("Commit")
except Exception as e:
        session.sql("Rollback")
        print(f"Error encountered: {e}. All inserts have been rolled back.")
    #print(insert_sql)